In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn import metrics
import xgboost as xgb
import datetime

pd.set_option('display.float_format',lambda x: '%.5f'%x)


In [2]:
print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close()

f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()

dfTrain = dfTrain.rename(columns=col_name)
dfTrain = dfTrain.replace({})
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfTrain = dfTrain.set_index(dfTrain['ID'])
dfPred = dfPred.rename(columns=col_name)
dfPred = dfPred.set_index(dfPred['ID'])
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)
dfPredAll = dfPredAll.set_index(dfPredAll['ID'])

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

Load data...
Done.


In [3]:
#load features
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')
   
        


In [ ]:
##define the variables
'''var_change = {'raw':raw}
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv',nrows=10)
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    var_change[filename] = {'constant':[],'category':[]}
    for var in tmp_feat_list:
        if dfTrain[var].nunique()==1 and dfTrain[var].isnull().sum()==0:
            var_change[filename]['constant'].append(var)
        elif dfTrain[var].nunique()<=5 or 'TOOL' in var:
            var_change[filename]['category'].append(var)
'''



In [4]:
#define the input variables, dummies
predictors = raw_predictors+ex_feat
feat_cnt = 0
for key,value in var_change.items():
    feat_cnt+=1
    if feat_cnt>1:
        continue
    for var in var_change[key]['constant']:
        predictors.remove(var)
    for var in var_change[key]['category']:
        predictors.remove(var)
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

In [26]:
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5, params = {'max_depth':4, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':2,'subsample':0.8,
                         'colsample_bytree':0.8}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param = params 
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =1000)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp,metrics.mean_squared_error(test_result['target'], test_result['score'])

In [28]:
record = []
for md in [3,4,5]:
    for la in [0.8,1,1.3,1.5,2]:
        for sp in [0.6,0.8,1]:
            for cb in [0.2,0.4,0.6,0.8,1]:
                tmp_record = [md,la,sp,cb]
                print(tmp_record)
                params = {'max_depth':md, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':la,'subsample':sp,'colsample_bytree':cb}
                tmp_record.append(xgb_kfold(dfTrain,dfPred,predictors,5,params=params)[3])
                record.append(tmp_record)
                GS_df = pd.DataFrame(record,columns=['max_depth','lambda','subsample','colsample_bytree','test_MSE'])
                GS_df.to_csv('../../Profiling/GS_result.csv',index=False)

[3, 0.8, 0.6, 0.2]
[0]	train-rmse:2.33435	eval-rmse:2.31993
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[508]	train-rmse:0.080587	eval-rmse:0.17733

[0]	train-rmse:2.32603	eval-rmse:2.35314
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[1000]	train-rmse:0.036267	eval-rmse:0.190875
Stopping. Best iteration:
[964]	train-rmse:0.038393	eval-rmse:0.19084

[0]	train-rmse:2.3309	eval-rmse:2.33339
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[626]	train-rmse:0.067672	eval-rmse:0.14085

[0]	train-rmse:2.32928	eval-rmse:2.34094
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved i

[0]	train-rmse:2.33139	eval-rmse:2.3318
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[654]	train-rmse:0.057253	eval-rmse:0.176195

[0]	train-rmse:2.3291	eval-rmse:2.34102
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[767]	train-rmse:0.044833	eval-rmse:0.182679

[0]	train-rmse:2.32968	eval-rmse:2.33864
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[717]	train-rmse:0.050013	eval-rmse:0.170166

[0]	train-rmse:2.3315	eval-rmse:2.33115
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[699]	train-rmse:0.0518	e

Stopping. Best iteration:
[785]	train-rmse:0.04275	eval-rmse:0.173689

[0]	train-rmse:2.32829	eval-rmse:2.34411
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[702]	train-rmse:0.051939	eval-rmse:0.179884

[0]	train-rmse:2.33696	eval-rmse:2.30894
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[490]	train-rmse:0.078336	eval-rmse:0.159276

[0]	train-rmse:2.32253	eval-rmse:2.36716
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[788]	train-rmse:0.045917	eval-rmse:0.178992

Test MSE: 0.0296774177999
[3, 0.8, 1, 0.8]
[0]	train-rmse:2.32872	eval-rmse:2.34241
Multiple eval metrics have been passed: 'eval-rmse' will be used for early st

Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[452]	train-rmse:0.089301	eval-rmse:0.176761

[0]	train-rmse:2.32897	eval-rmse:2.34168
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[563]	train-rmse:0.070801	eval-rmse:0.157543

[0]	train-rmse:2.33728	eval-rmse:2.30753
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[773]	train-rmse:0.047698	eval-rmse:0.188301

Test MSE: 0.0297359455821
[3, 1, 0.6, 1]
[0]	train-rmse:2.32764	eval-rmse:2.34783
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[812]	train-rmse:0.045

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[556]	train-rmse:0.065903	eval-rmse:0.15246

[0]	train-rmse:2.3327	eval-rmse:2.32606
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[751]	train-rmse:0.046035	eval-rmse:0.176277

Test MSE: 0.0291668456832
[3, 1, 1, 0.2]
[0]	train-rmse:2.33706	eval-rmse:2.30854
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[524]	train-rmse:0.074444	eval-rmse:0.182298

[0]	train-rmse:2.32441	eval-rmse:2.3597
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[817]	train-rmse:0.046371	eval-rmse:0.182413

[0]	train-rmse:2.32815	eval-rmse:2.34475
Multiple eval metrics ha

...

[0]	train-rmse:2.33211	eval-rmse:2.3287
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[507]	train-rmse:0.032443	eval-rmse:0.176651

[0]	train-rmse:2.33632	eval-rmse:2.31154
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[541]	train-rmse:0.026333	eval-rmse:0.173241

Test MSE: 0.0306876682212
[5, 1, 0.6, 0.2]
[0]	train-rmse:2.33308	eval-rmse:2.32497
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[610]	train-rmse:0.033814	eval-rmse:0.170455

[0]	train-rmse:2.32537	eval-rmse:2.35642
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stoppi

[0]	train-rmse:2.32928	eval-rmse:2.34027
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[1000]	train-rmse:0.00659	eval-rmse:0.161287
Stopping. Best iteration:
[991]	train-rmse:0.006809	eval-rmse:0.161267

[0]	train-rmse:2.33795	eval-rmse:2.30491
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[443]	train-rmse:0.054516	eval-rmse:0.174929

Test MSE: 0.0292905229551
[5, 1, 0.8, 0.4]
[0]	train-rmse:2.33486	eval-rmse:2.31731
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[897]	train-rmse:0.009037	eval-rmse:0.178729

[0]	train-rmse:2.33227	eval-rmse:2.32814
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until 

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[713]	train-rmse:0.013178	eval-rmse:0.174386

[0]	train-rmse:2.3287	eval-rmse:2.34254
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[1000]	train-rmse:0.005159	eval-rmse:0.152358
Stopping. Best iteration:
[1285]	train-rmse:0.001941	eval-rmse:0.152241

[0]	train-rmse:2.33825	eval-rmse:2.30369
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[424]	train-rmse:0.055953	eval-rmse:0.1739

Test MSE: 0.0305986136168
[5, 1, 1, 0.6]
[0]	train-rmse:2.3331	eval-rmse:2.32473
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[617]	train-rmse:0.02044	eval-rmse:0.170423

[0]	train-rmse:2.330

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[901]	train-rmse:0.015031	eval-rmse:0.164602

[0]	train-rmse:2.32882	eval-rmse:2.34204
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[925]	train-rmse:0.013886	eval-rmse:0.188822

[0]	train-rmse:2.34184	eval-rmse:2.28937
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[505]	train-rmse:0.051016	eval-rmse:0.160799

Test MSE: 0.0297166919983
[5, 1.3, 0.6, 0.8]
[0]	train-rmse:2.33599	eval-rmse:2.31302
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[526]	train-rmse:0.047238	eval-rmse:0.161765

[0]	train-rmse:2.33586	eval-rmse:2.31428
Multiple eval met

KeyboardInterrupt: 

In [12]:
test_result,result,imp = xgb_kfold(dfTrain,dfPred,predictors)

[0]	train-rmse:2.33325	eval-rmse:2.32446
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.42504	eval-rmse:1.41672
[100]	train-rmse:0.879138	eval-rmse:0.872982
[150]	train-rmse:0.55208	eval-rmse:0.555279
[200]	train-rmse:0.355857	eval-rmse:0.372674
[250]	train-rmse:0.236805	eval-rmse:0.272561
[300]	train-rmse:0.163235	eval-rmse:0.220306
[350]	train-rmse:0.117279	eval-rmse:0.19572
[400]	train-rmse:0.088505	eval-rmse:0.184687
[450]	train-rmse:0.069554	eval-rmse:0.180748
[500]	train-rmse:0.056817	eval-rmse:0.178809
[550]	train-rmse:0.047473	eval-rmse:0.17801
[600]	train-rmse:0.04041	eval-rmse:0.177439
[650]	train-rmse:0.034622	eval-rmse:0.17706
[700]	train-rmse:0.029927	eval-rmse:0.177028
[750]	train-rmse:0.025907	eval-rmse:0.176934
[800]	train-rmse:0.022595	eval-rmse:0.176859
[850]	train-rmse:0.019805	eval-rmse:0.176851
Stopping. Best iteration:
[819]	train-rmse:0.021476	eval-rm

In [13]:
imp.to_csv('../../Profiling/imp_test_v3.csv',index=False)

In [ ]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)